| Task| Student | Teacher | Date| 
|----------|----------|----------|--------|
|Pseudo Object Detection Code Correction| Lovisa Nguyen | Toni Aaltonen |11/23|

In [22]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

def BLUR(image):
    # Transfrom image to its gray-version for a better highlight of edges
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Blurring Filter
    # Scanning using kernel_blurring by taking (average of each pixel)/9 with its neigbour ones
    kernel_blurring = np.array([
                                [1,1,1],
                                [1,1,1],
                                [1,1,1], 
                               ])/9 
    image_blurred = cv2.filter2D(image_gray, -1, kernel_blurring)
    # Edge Detection via Gradient
    image_gray_float = image_blurred #.astype(np.float32)
    return image_gray_float


def ORB_detector(new_image, image_template):
    edges0 = cv2.Canny(new_image,10,50, cv2.COLOR_BGR2GRAY)
    edges1 = cv2.Canny(image_template,10,50, cv2.COLOR_BGR2GRAY)
    
    orb = cv2.ORB_create(1000, 1.2)  # ORB detector of 1000 keypoints, scaling pyramid factor=1.2
    (kp1, des1) = orb.detectAndCompute(edges0, None)  # Detect keypoints on the new image
    (kp2, des2) = orb.detectAndCompute(edges1, None)  # Detect keypoints of the template image
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)  # Matcher
    matches = bf.match(des1, des2)  # Extract matches
    matches = sorted(matches, key=lambda x: x.distance)  # Sort matches
    #img2 = cv2.drawKeypoints(edges1, kp2, None, color=(0,255,0), flags=0)
    #img1 = cv2.drawKeypoints(edges0, kp1, None, color=(0,255,0), flags=0)
    #plt.imshow(img1)
    #plt.show()
    return len(matches)

# Load video file and template image
cap = cv2.VideoCapture("HethenHisPelivideo.mp4")
image_template = cv2.imread('HisPelivideo.png', 0)


# Initialize ROI coordinates
top_left_x, top_left_y, bottom_right_x, bottom_right_y = 0, 0, 0, 0

while (cap.isOpened()):
    # Get video frame
    ret, frame = cap.read()

    # Check if the video frame was captured successfully
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert frame to grayscale for ORB detector
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Get number of ORB matches
    matches = ORB_detector(gray_frame, image_template)

    # Display status string showing the current number of matches
    output_string = "Matches = " + str(matches)
    cv2.putText(frame, output_string, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

    # Our threshold to indicate object detection
    threshold = 337

    # If matches exceed our threshold then object has been detected
    if matches > threshold:
        # Perform template matching to get the location of the detected object
        result = cv2.matchTemplate(gray_frame, image_template, cv2.TM_CCOEFF_NORMED)
        _, _, _, max_loc = cv2.minMaxLoc(result)
        top_left_x, top_left_y = max_loc
        bottom_right_x = top_left_x + image_template.shape[1]
        bottom_right_y = top_left_y + image_template.shape[0]
        
        # Resize the frame region to match the detected object's size
        detected_object_width = bottom_right_x - top_left_x
        detected_object_height = bottom_right_y - top_left_x
        # Update the top-left and bottom-right coordinates of the detection frame
        detection_frame_width = detected_object_width
        detection_frame_height = detected_object_height
        # Draw a rectangle around the detected object with the resized frame size
        cv2.rectangle(frame, (int(top_left_x), int(top_left_y)),
                      (int(top_left_x + detection_frame_width), int(top_left_y + detection_frame_height)),
                      (0, 255, 0), 3)
        cv2.putText(frame, 'Object Found', (50, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

    # Display the frame with the detected object
    cv2.imshow('Object Detector using ORB', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

In [25]:
### Main

import numpy as np
import cv2
from matplotlib import pyplot as plt

def ORB_detector(new_image, image_template):
    edges0 = cv2.Canny(new_image,10,50, cv2.COLOR_BGR2GRAY)
    edges1 = cv2.Canny(image_template,10,50, cv2.COLOR_BGR2GRAY)
    
    orb = cv2.ORB_create(1000, 1.2)  # ORB detector of 1000 keypoints, scaling pyramid factor=1.2
    (kp1, des1) = orb.detectAndCompute(edges0, None)  # Detect keypoints on the new image
    (kp2, des2) = orb.detectAndCompute(edges1, None)  # Detect keypoints of the template image
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)  # Matcher
    matches = bf.match(des1, des2)  # Extract matches
    matches = sorted(matches, key=lambda x: x.distance)  # Sort matches
    #img2 = cv2.drawKeypoints(edges1, kp2, None, color=(0,255,0), flags=0)
    #img1 = cv2.drawKeypoints(edges0, kp1, None, color=(0,255,0), flags=0)
    #plt.imshow(img1)
    #plt.show()
    return len(matches)

cap = cv2.VideoCapture("HethenHisPelivideo.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
target_time = 3
init_frame = int(target_time * fps)
ret, initial_frame = cap.read()
gray_initial_frame = cv2.cvtColor(initial_frame, cv2.COLOR_BGR2GRAY)

# Initialize ROI coordinates
top_left_x, top_left_y, bottom_right_x, bottom_right_y = 0, 0, 0, 0



while True:
    # Get webcam images
    ret, frame = cap.read()

    # Get height and width of webcam frame
    height, width = frame.shape[:2]

    # Define ROI Box Dimensions (Note some of these things should be outside the loop)
    top_left_x = int(width / 3)
    top_left_y = int((height / 2) + (height / 4))
    bottom_right_x = int((width / 3) * 2)
    bottom_right_y = int((height / 2) - (height / 4))

    # Draw rectangular window for our region of interest
    # cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), 255, 3)

    # Crop window of observation we defined above
    cropped = frame[bottom_right_y:top_left_y , top_left_x:bottom_right_x]

    # Flip frame orientation horizontally
    frame = cv2.flip(frame,1)

    # Get number of ORB matches 
    matches = ORB_detector(cropped, image_template)

    # Display status string showing the current no. of matches 
    output_string = "Matches = " + str(matches)
    cv2.putText(frame, output_string, (50,450), cv2.FONT_HERSHEY_COMPLEX, 2, (250,0,150), 2)

    # Our threshold to indicate object deteciton
    # For new images or lightening conditions you may need to experiment a bit 
    # Note: The ORB detector to get the top 1000 matches, 350 is essentially a min 35% match
    threshold = 336

    # If matches exceed our threshold then object has been detected
    if matches > threshold:
        # Update the initial frame and keypoints for the next iteration
        initial_frame = frame
        kp1 = kp2

        # Draw the detecting frame around the object using the matched keypoints
        # Example: 
        for match in matches[:10]:
            x, y = kp1[match.queryIdx].pt
            cv2.rectangle(frame, (int(x + 10), int(y + 10)), (int(x + 100), int(y + 100)), (0, 255, 0), 2)
        cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,255,0), 3)
        cv2.putText(frame,'Object Found',(50,50), cv2.FONT_HERSHEY_COMPLEX, 2 ,(0,255,0), 2)
        # Display the frame with the detected object
        cv2.imshow('Object Detector using ORB', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

TypeError: 'int' object is not subscriptable

In [4]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

import cv2
import numpy as np

def get_background(file_path):
    cap = cv2.VideoCapture(file_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Handle frames with different dimensions, if necessary
        # frame = cv2.resize(frame, (desired_width, desired_height))
        frames.append(frame)
    cap.release()  # Release the video capture object
    if len(frames) > 0:
        median_frame = np.median(frames, axis=0).astype(np.uint8)
        return median_frame
    else:
        # Handle the case when no valid frames are read
        print("Error: No valid frames were read from the video.")
        return None

# Example usage
background = get_background("video_1.mp4")

if background is not None:
    print("Background data type:", background.dtype)
    background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    print("Background data type after conversion:", background_gray.dtype)
else:
    print("Error: No valid background frame.")



def ORB_detector(new_image, image_template):
    orb = cv2.ORB_create(1000, 1.2, 2, 31)  # ORB detector of 1000 keypoints, scaling pyramid factor=1.2
    (kp1, des1) = orb.detectAndCompute(new_image, None)  # Detect keypoints on the new image
    (kp2, des2) = orb.detectAndCompute(image_template, None)  # Detect keypoints of the template image
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)  # Matcher
    matches = bf.match(des1, des2)  # Extract matches
    matches = sorted(matches, key=lambda val: val.distance)  # Sort matches
    img2 = cv2.drawKeypoints(image_template, kp2, None, color=(0,255,0), flags=0)
    img1 = cv2.drawKeypoints(new_image, kp2, None, color=(0,255,0), flags=0)
    plt.imshow(img2)
    plt.show()
    return len(matches)

# Load video file and template image

consecutive_frames = 500

background = get_background("video_1.mp4")
print("Background data type:", background.dtype)
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
print("Background data type:", background.dtype)


frame_count = 0
frame_diff_list = []

while True:
    cap = cv2.VideoCapture("video_1.mp4")
    ret, frame = cap.read()

    frame_count += 1
    orig_frame = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.uint8)
    print("Background shape:", background.shape)
    print("Gray shape:", gray.shape)

    frame_diff = cv2.absdiff(gray, background_gray)
    ret, thres = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)
    dilate_frame = cv2.dilate(thres, None, iterations=2)
    frame_diff_list.append(dilate_frame)
    if len(frame_diff_list) == consecutive_frames:
        sum_frames = sum(frame_diff_list)
        contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if cv2.contourArea(contour) < 500:
                continue
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(orig_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Get height and width of webcam frame
        height, width = frame.shape[:2]

        # Define ROI Box Dimensions (Note some of these things should be outside the loop)
        top_left_x = int(width / 3)
        top_left_y = int((height / 2) + (height / 4))
        bottom_right_x = int((width / 3) * 2)
        bottom_right_y = int((height / 2) - (height / 4))

        # Draw rectangular window for our region of interest
        cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), 255, 3)

        # Crop window of observation we defined above
        cropped = frame[bottom_right_y:top_left_y , top_left_x:bottom_right_x]

        # Flip frame orientation horizontally
        frame = cv2.flip(frame,1)

        # Get number of ORB matches
        matches = ORB_detector(cropped, image_template)

        # Display status string showing the current no. of matches
        output_string = "Matches = " + str(matches)
        cv2.putText(frame, output_string, (50,450), cv2.FONT_HERSHEY_COMPLEX, 2, (250,0,150), 2)

        # Our threshold to indicate object deteciton
        # For new images or lightening conditions you may need to experiment a bit
        # Note: The ORB detector to get the top 1000 matches, 350 is essentially a min 35% match
        threshold = 336

        # If matches exceed our threshold then object has been detected
        if matches > threshold:
            cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,255,0), 3)
            cv2.putText(frame,'Object Found',(50,50), cv2.FONT_HERSHEY_COMPLEX, 2 ,(0,255,0), 2)
            # Display the frame with the detected object
        cv2.imshow('Object Detector using ORB', frame)

        cv2.imshow('Detected Objects', orig_frame)
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

Background data type: uint8
Background data type after conversion: uint8
Background data type: uint8
Background data type: uint8
Background shape: (360, 640, 3)
Gray shape: (360, 640)


In [3]:
# Work on this, deeper into the ORB

import cv2

def ORB_detector(new_image, image_template):
    orb = cv2.ORB_create(1000, 1.2, 2, 31)  # ORB detector of 1000 keypoints, scaling pyramid factor=1.2
    (kp1, des1) = orb.detectAndCompute(new_image, None)  # Detect keypoints on the new image
    (kp2, des2) = orb.detectAndCompute(image_template, None)  # Detect keypoints of the template image
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)  # Matcher
    matches = bf.match(des1, des2)  # Extract matches
    matches = sorted(matches, key=lambda val: val.distance)  # Sort matches
    img2 = cv2.drawKeypoints(image_template, kp2, None, color=(0,255,0), flags=0)
    img1 = cv2.drawKeypoints(new_image, kp2, None, color=(0,255,0), flags=0)
    plt.imshow(img2)
    plt.show()
    return len(matches)

cap = cv2.VideoCapture("HethenHisPelivideo.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
target_time = 3
init_frame = int(target_time * fps)
ret, initial_frame = cap.read()
gray_initial_frame = cv2.cvtColor(initial_frame, cv2.COLOR_BGR2GRAY)
while True:
    
    
    
    ret, frame = cap.read()
    
    
    
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(gray_initial_frame, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    ret, frame = cap.read()
    if not ret:
        break

    # Detect and compute ORB keypoints and descriptors for the current frame
    kp2, des2 = orb.detectAndCompute(frame, None)

    # Match descriptors using the BFMatcher
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    
    
    # Draw matches on the frames (optional)
    # img_matches = cv2.drawMatches(initial_frame, kp1, frame, kp2, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    # Update the initial frame and keypoints for the next iteration
    initial_frame = frame
    kp1 = kp2
    
    threshold = 341
    # Draw the detecting frame around the object using the matched keypoints
    # Example: 
    
    for match in matches[:50]:
        x, y = kp1[match.queryIdx].pt
        cv2.rectangle(frame, (int(x+20), int(y+20)), (int(x+10), int(y+10)), (0, 255, 0), 2)
        
        
        # Define the center, axes lengths, angle, and color for the ellipse
        center = (int(x), int(y))  # (x, y) coordinates of the center
        axes_lengths = (50, 20)  # Lengths of major and minor axes (adjust as needed)
        angle = 0  # Rotation angle in degrees
        start_angle = 0  # Start angle of the ellipse arc in degrees
        end_angle = 360  # End angle of the ellipse arc in degrees
        color = (0, 255, 0)  # Green color in BGR

        # Define the thickness of the ellipse outline (-1 for filled ellipse)
        thickness = 2

        # Draw the ellipse around the point (x, y)
        cv2.ellipse(frame, center, axes_lengths, angle, start_angle, end_angle, color, thickness)

        # Draw the smaller rectangle around the point (x, y)
        cv2.rectangle(frame, (int(x + 20), int(y + 20)), (int(x + 10), int(y + 10)), (0, 255, 0), 2)

        # Show the image with the ellipse and rectangle
        cv2.imshow("Ellipse and Rectangle", frame)
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()
